In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import io

import numpy as np
import pymongo

from bokeh.io import output_notebook, show
from bokeh.palettes import all_palettes
from bokeh.layouts import gridplot

from IPython import display

from climatools.cliradlw.dataset import load_output_file
import climatools.html.html as climahtml
from climatools.plot.plot import *

In [3]:
output_notebook()

Loading BokehJS ...

In [4]:
def make_query(param=None):
    '''
    Returns the MongoDB query for a lblnew 
    run's document.
    
    Parameters
    ----------
    param: dict
    lblnew-bestfit input parameters.
    '''
    return {'param.' + name: value for name, value in param.items()}   

In [5]:
def print_input_param():
    '''
    Print to screen the input parameters 
    of the lblnew run.
    '''
    df = pd.DataFrame(columns=['lblnew'])
    for name, value in sorted(PARAM.items()):
        df.loc[name, 'lblnew'] = value
    
    df.index.name = 'Input parameter'
    df.columns.name = 'Run'
    
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
        
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band') 
            
    return ds['coolrg']



def pltdata_cool(g=None):
    '''
    Define the plotting data for cooling rate
    profile plots.  Add/remove entries in
    the `data` list.
    
    Parameters
    ----------
    g: int
        g-interval to be plotted.  If None,
        the sum over all g-intervals will be
        plotted.
    data: dict
        Plot data, including colour, line attributes, etc.
    '''
    colors = all_palettes['Set1'][4]
    
    if g == None:
        ds_cool_crd = DS_COOL_CRD.sum('g')
        ds_cool_wgt = DS_COOL_WGT.sum('g')
    else:
        ds_cool_crd = DS_COOL_CRD.sel(g=g)
        ds_cool_wgt = DS_COOL_WGT.sel(g=g)
    
    data = [
        {'label': 'CRD',
         'srs': fmt_cool(ds_cool_crd),
         'line_dash': 'solid', 'line_width':1.5,
         'marker': 'circle', 'mark_size': 5,
         'color': colors[2], 'alpha': 1},
        {'label': 'WGT igg=1',
         'srs': fmt_cool(ds_cool_wgt.sel(igg=1)),
         'line_dash': 'solid', 'line_width': 3,
         'marker': 'square', 'marker_size': 5,
         'color': colors[3], 'alpha': .6},
        {'label': 'WGT igg=10',
         'srs': fmt_cool(ds_cool_wgt.sel(igg=10)),
         'line_dash': 'solid', 'line_width': 2,
         'marker': 'triangle', 'marker_size': 2,
         'color': colors[1], 'alpha': .3}]
    return data



def plt_cool(pltdata=None):
    '''
    Plot cooling rate profiles on linear
    and log pressure axes.
    
    Parameters
    ----------
    pltdata: dict
        Plot data, including colour, line attributes, etc.
    everything: bokeh.models.layouts.Column
        Cooling rate profile plot.
    '''
    plin = plt_vert_profile_bokeh(pltdata=pltdata, 
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200)) 
    plog.plot_width, plog.plot_height = 400, 500
    
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
    

def script_plt_cool():
    '''
    Script to plot the cooling rate profiles
    in each g-interval and the total 
    cooling rate profile over all g-intervals.
    '''
    ng = sum(PARAM['ng_refs'])
    for g in np.arange(1, ng + 1):
        data = pltdata_cool(g=g)
        p = plt_cool(pltdata=data)
        show(p)
        display.display(
            display.Markdown(f'*FIGURE.* Cooling rates. g={g}'))
    
    data = pltdata_cool()
    p = plt_cool(pltdata=data)
    show(p)
    display.display(
        display.Markdown(f'*FIGURE.* Cooling rates. Summed over g.'))
    


In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    #if 'igg' in ds.dims:
    #    ds = ds.squeeze('igg').drop('igg')
        
    if 'igg' in ds:
        if ds['igg'].shape == (1,):
            ds = ds.squeeze('igg').drop('igg')
        elif ds['igg'].shape == ():
            ds = ds.drop('igg')
            
    if 'g' in ds:
        if ds['g'].shape == (1,):
            ds = ds.squeeze('g').drop('g')
        elif ds['g'].shape == ():
            ds = ds.drop('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux():
    '''
    Define the flux data to be tabulated for
    the g-intervals.  Add/remove entries in the
    `tbs` list.
    
    Parameters
    ----------
    data: dict
        Flux data, including how each set is to be
        labeled in the output table.
    '''
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=1',
            'df': fmt_flux(DS_FLUX_WGT.sel(igg=[1]), ilevels=ilevels)},
           {'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sel(igg=[10]), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data



def tbdata_flux_sumg():
    '''
    Define the total flux data, over all g-intervals,
    to be tabulated.
    
    Parameters
    ----------
    data: dict
        Flux data, including how each set is to e
        labeled in the output table.
    '''
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=1',
            'df': fmt_flux(DS_FLUX_WGT.sel(igg=[1]).sum('g'), 
                           ilevels=ilevels)},
           {'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sel(igg=[10]).sum('g'), 
                           ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    '''
    Display flux table for each set, and display
    the difference between each set and the set
    that is specified to be the benchmark.
    
    Parameters
    ----------
    tbdata: dict
        Flux data, including how each set it to be
        labeled in the output table.
    '''
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        display.display(display.Markdown(
            f"*TABLE.* Fluxes. {d['label']}"))
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        display.display(display.Markdown(
            f"*TABLE.* Fluxes. ({d['label']}) - ({d0['label']})"))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        

        
def script_tb_flux():
    '''
    Script to display the flux tables in 
    each g-interval, and the flux table
    for the total flux, over all g-intervals.
    '''
    tb_flux(tbdata=tbdata_flux())
    tb_flux(tbdata=tbdata_flux_sumg())


In [8]:
def script_gsummary():
    '''
    Script to display how the g-intervals
    are defined according the hyperparameters
    chosen for this lblnew-bestfit run.
    '''
    ng_refs = PARAM['ng_refs']
    p_refs = list(zip(*PARAM['ref_pts']))[0]
    ng = sum(ng_refs)
    df_abscom = pd.read_csv(io.StringIO(DOC['abscom']), 
                            sep=r'\s+', index_col=[0])
    df_dgdgs = pd.read_csv(io.StringIO(DOC['dgdgs']), 
                            sep=r'\s+', index_col=[0])
    
    # absth(g + 1) / absth(g)
    df_abscom_copy = df_abscom.copy()
    df_abscom_copy.index = df_abscom.index + 1
    df_absratio = df_abscom / df_abscom_copy
    df_absratio.columns = ['k(ig) / k(ig - 1)']
    df_absratio = df_absratio.loc[: ng]    
    
    df_dgdgs_cumsum = df_dgdgs.cumsum(axis=0)
    df_dgdgs_cumsum.columns = ['dg cumsum', 'dgs cumsum']
    
    data_pref = np.concatenate([p_ref * np.ones((ng_ref,)) 
                                for ng_ref, p_ref 
                                in zip(ng_refs, p_refs)], axis=0)
    df_pref = pd.DataFrame(data_pref, 
                           index=df_dgdgs.index, columns=['pref'])
    
    df_all = pd.concat([df_abscom, df_absratio,
                        df_dgdgs, df_dgdgs_cumsum,
                        df_pref], axis=1) 
    
    display.display(df_all)
    display.display(display.Markdown(
        (f"TABLE. g-interval summary.  "
         f"*abscom* is the lower bound of the g-interval.")))
    


In [9]:
def script_flux_to_excel():
    '''
    Script to save the lblnew-bestfit run's
    output flux to excel file and display the
    link to download the excel file.
    '''
    fname = 'lblnew-bestfit_flux.xlsx'
    writer = pd.ExcelWriter(fname)
    
    ds = DS_FLUX_CRD
    for g in ds.coords['g']:
        df = ds.sel(g=g).to_dataframe()
        df.to_excel(writer, f'crd flux g={int(g)}')
                
    ds = DS_FLUX_WGT
    for g in ds.coords['g']:
        df = ds.sel(g=g, igg=1).to_dataframe()
        df.to_excel(writer, f'wgt igg=1 flux g={int(g)}')
        
        df = ds.sel(g=g, igg=10).to_dataframe()
        df.to_excel(writer, f'wgt igg=10 flux g={int(g)}')
        
    writer.save()    
    html_download = f'Download: <a href="./{fname}">{fname}</a>'
    display.display(display.HTML(html_download))


def script_cool_to_excel():
    '''
    Script to save the lblnew-bestfit run's
    output cooling rate to excel file and display
    the link to download the excel file.
    '''
    fname = 'lblnew-bestfit_cool.xlsx'
    writer = pd.ExcelWriter(fname)
    
    ds = DS_COOL_CRD
    for g in ds.coords['g']:
        df = ds.sel(g=g).to_dataframe()
        df.to_excel(writer, f'crd cool g={int(g)}')
        
    ds = DS_COOL_WGT
    for g in ds.coords['g']:
        df = ds.sel(g=g, igg=1).to_dataframe()
        df.to_excel(writer, f'wgt igg=1 cool g={int(g)}')
        
        df = ds.sel(g=g, igg=10).to_dataframe()
        df.to_excel(writer, f'wgt igg=10 cool g={int(g)}')
        
    writer.save()
    html_download = f'Download: <a href="./{fname}">{fname}</a>'
    display.display(display.HTML(html_download))

In [10]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

In [11]:
def script():
    title = ('### lblnew-bestfit' 
             if 'ng_refs' in PARAM else '### lblnew-overlap')
    s_input = 'Input Parameters'
    s_plt_cool = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_gsummary = 'g-interval Summary'
    s_download = 'Download Data'
    
    # title
    show_markdown(title)
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cool))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_gsummary))
    show_html(climahtml.getHTML_hrefanchor(s_download))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cool))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cool))
    script_plt_cool()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # g-interval summary
    show_html(climahtml.getHTML_idanchor(s_gsummary))
    show_markdown(climahtml.getMarkdown_sectitle(s_gsummary))
    script_gsummary()
    
    # Download cooling rate and fluxes
    show_html(climahtml.getHTML_idanchor(s_download))
    show_markdown(climahtml.getMarkdown_sectitle(s_download))
    script_flux_to_excel()
    script_cool_to_excel()

In [12]:
%run param.py

client = pymongo.MongoClient('localhost', 27017, connect=False)
db = client.lblnew


DOC = db.bestfit_lw.find_one(make_query(param=PARAM))
DS_FLUX_CRD = load_output_file(io.StringIO(DOC['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(DOC['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(DOC['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(DOC['output_wcoolrg']))

In [13]:
script()

### lblnew-bestfit

### Table of Contents

# Input Parameters

Run,lblnew
Input parameter,
atmpro,saw
band,6
commitnumber,5014a19
conc,None
dv,0.001
klin,5e-25
molecule,h2o
ng_adju,[0]
ng_refs,[4]


# Cooling Rate Profiles

...

*TABLE.* Fluxes. (WGT igg=10) - (CRD)

# g-interval Summary

,abscom,k(ig) / k(ig - 1),dg,dgs,dg cumsum,dgs cumsum,pref
g,,,,,,,
1,1.089444e-22,NaN,0.015913,0.014863,0.015913,0.014863,600.0
2,7.380882e-24,0.067749,0.089487,0.086290,0.105400,0.101153,600.0
3,5.000477e-25,0.067749,0.357470,0.344110,0.462870,0.445263,600.0
4,3.773524e-26,0.075463,0.537130,0.554737,1.000000,1.000000,600.0


TABLE. g-interval summary.  *abscom* is the lower bound of the g-interval.

# Download Data

In [14]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')